In [ ]:
import qsharp

In [ ]:
%%qsharp
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Measurement;

operation ApplyOracle(inputQubits : Qubit[], xPrime : Int[]) : Unit {
    within {
        for i in 0..Length(xPrime)-1 {
            if xPrime[i] == 0 {
                X(inputQubits[i]);
            }
        }
    } apply {
        Controlled Z(Most(inputQubits), Tail(inputQubits));
    }
}

operation Diffuser(inputQubits : Qubit[]) : Unit {
    within {
        Adjoint ApplyToEachCA(H, inputQubits);
        ApplyToEachCA(X, inputQubits);
    } apply {
        Controlled Z(Most(inputQubits), Tail(inputQubits));
    }
}

operation GroversAlgorithm(nQubits : Int, xPrime : Int[]) : Result[] {
    use qubits = Qubit[nQubits];
    ApplyToEachCA(H, qubits);
    for idxIteration in 0..(Round(0.25 * PI() / ArcSin(1. / Sqrt(IntAsDouble(1 <<< nQubits))) - 0.5) - 1) {
        ApplyOracle(qubits, xPrime);
        Diffuser(qubits);
    }
    return ForEach(MResetZ, qubits);
}

In [ ]:
import time
import random
import numpy as np
def calculate_time(n, xprime):
        start = time.time()
        result = GroversAlgorithm(nQubits=n, xPrime=xprime)
        end = time.time()
        return end - start, result

sum_times = [0] * 30
for i in range(5):
    times = []
    for n in range(2, 32):
        xprime = [random.randint(0, 1) for _ in range(n)]
        no_iterations = int(np.floor(np.pi / 4 * np.sqrt(2 ** n)))
        print(f"for {n} qubits:")
        try:
            elapsed_time, result = calculate_time(n, xprime)
            times.append(elapsed_time)
            result = int("".join(str(x) for x in result)[::-1], base=2)
            print(f"took {elapsed_time}s, with x'={xprime} and {no_iterations} iterations, found={result == xprime}")
        except Exception as e:
            print(str(e))
            break
    for j in range(30):
        sum_times[j] += times[j]
avg_times = [t / 5 for t in sum_times]
print(avg_times)